In [1]:
import json  
import os, sys, pathlib
import pickle

import pandas as pd
import matplotlib.pyplot as plt
import cv2

import importlib

BASE_DIR = pathlib.Path.cwd().parent


In [2]:
sys.path.append(str(BASE_DIR / 'src'))

import ollama_manager as om
import main
import STAR_utils.visualization_tools.qa_visualization as qaviz


In [3]:
# please replace following data path to your local path
# all data can be download from our homepages (http://star.csail.mit.edu or https://bobbywu.com/STAR)

# root_dir = '/root/user/'
raw_video_dir = BASE_DIR / 'data/datasets/action-genome/Charades_v1_480/'
raw_frame_dir = BASE_DIR / 'data/datasets/action-genome/frames/'
annotation_dir = BASE_DIR / 'src/STAR_utils/annotations/STAR_classes/'
# pose_dir = root_dir + 'STAR/pose/'
# qa_train_dir = BASE_DIR /'data/datasets/STAR/STAR_annotations/STAR_train.json' 
qa_val_data = BASE_DIR / 'data/datasets/STAR/STAR_annotations/STAR_val.json' 
# qa_test_dir = BASE_DIR / 'data/datasets/STAR/STAR_annotations/STAR_test.json' 
save_video_dir = BASE_DIR / 'visualization_tmp'
fps = pickle.load(open(annotation_dir / 'video_fps','rb'))


In [6]:
star_data = []
with open(qa_val_data, 'r') as f:
    star_data = json.load(f)


In [ ]:
star_df = pd.DataFrame(star_data)
star_df = star_df.set_index('question_id')
star_df


,question,video_id,start,end,answer,question_program,choices,situations
question_id,,,,,,,,
Interaction_T1_13,Which object was tidied up by the person?,6H78U,11.1,19.6,The clothes.,"[{'function': 'Situations', 'value_input': []}...","[{'choice_id': 0, 'choice': 'The closet/cabine...","{'000206': {'rel_pairs': [['o000', 'o027'], ['..."
Interaction_T1_14,Which object was tidied up by the person?,6H78U,15.6,22.7,The clothes.,"[{'function': 'Situations', 'value_input': []}...","[{'choice_id': 0, 'choice': 'The blanket.', 'c...","{'000289': {'rel_pairs': [['o000', 'o027'], ['..."
Interaction_T1_31,Which object was thrown by the person?,RNLTR,7.3,13.5,The clothes.,"[{'function': 'Situations', 'value_input': []}...","[{'choice_id': 0, 'choice': 'The pillow.', 'ch...","{'000132': {'rel_pairs': [['o000', 'o019'], ['..."
Interaction_T1_32,Which object was put down by the person?,RNLTR,11.6,16.4,The shoe.,"[{'function': 'Situations', 'value_input': []}...","[{'choice_id': 0, 'choice': 'The food.', 'choi...","{'000202': {'rel_pairs': [['o000', 'o019'], ['..."
Interaction_T1_40,Which object was tidied up by the person?,VNQTH,2.7,9.2,The closet/cabinet.,"[{'function': 'Situations', 'value_input': []}...","[{'choice_id': 0, 'choice': 'The broom.', 'cho...","{'000110': {'rel_pairs': [['o000', 'o014'], ['..."
...,...,...,...,...,...,...,...,...
Feasibility_T6_1453,What is the person able to do after taking the...,L9ANI,7.3,12.9,Wash the table.,"[{'function': 'Situations', 'value_input': []}...","[{'choice_id': 0, 'choice': 'Wash the table.',...","{'000227': {'rel_pairs': [['o000', 'o006'], ['..."
Feasibility_T6_1454,What is the person able to do after walking th...,L9ANI,0.0,4.1,Take the towel.,"[{'function': 'Situations', 'value_input': []}...","[{'choice_id': 0, 'choice': 'Take the towel.',...","{'000033': {'rel_pairs': [['o000', 'o006']], '..."
Feasibility_T6_1455,What is the person able to do after walking th...,L9ANI,0.0,4.1,Wash the table.,"[{'function': 'Situations', 'value_input': []}...","[{'choice_id': 0, 'choice': 'Throw the bag.', ...","{'000033': {'rel_pairs': [['o000', 'o006']], '..."


In [30]:
sorted(star_df.loc['Interaction_T1_14']['situations'].keys())


['000286',
 '000289',
 '000294',
 '000301',
 '000303',
 '000308',
 '000313',
 '000314',
 '000316',
 '000318',
 '000320',
 '000322',
 '000331',
 '000332',
 '000336',
 '000337',
 '000339',
 '000364',
 '000370',
 '000377',
 '000388',
 '000389',
 '000392']

In [35]:
import main

SEED = 712407852679

url = os.environ.get("OLLAMA_URL", "http://lusha_ollama:11435")

sys_prompt = main._load_prompt_fromfile(
        BASE_DIR / "data/prompts/zero-shot-cot/MCQ_system_prompt_ZS_CoT.txt"
    )

ollama_params={
            "model": "gemma3:4b",
            "system": None,
            "stream": True,
            "options": {
                "num_ctx": 10240,
                "temperature": 0.1,
                "num_predict": 1024,
                "seed": SEED,
            },
        }

client = om.OllamaRequestManager(url, ollama_params)


In [34]:
import importlib

importlib.reload(main)


<module 'main' from '/home/lusha/star_code/src/main.py'>

In [36]:
import prompt_formatters as pf

user_prompt = '''\
You will receive a sequence of images and a question related to them. Analyze the images in the order they are presented,\
extract relevant details, and identify any patterns, changes, or relationships between them. Use this information to provide\
a choose one of the presented alternatives

Instructions:
- Describe each image in the sequence briefly but meaningfully.
- Note any important transitions or developments between consecutive images.
- Combine observations to infer the overall context or narrative.
- Answer the question based on your analysis, ensuring your response is grounded in the visual evidence.
- Read the question and reason about the answer step by step.
- In your answer include key events or relationships that help you in determine the correct answer.
- Be careful to reproduce the chosen alternative as it is presented.

Q: {question}
<Alternatives>
A. {c1}
B. {c2}
C. {c3}
D. {c4}
</Alternatives>\
'''

mcq_pformatter = pf.MCQPromptWoutSTSG(user_prompt)
input_filepath = BASE_DIR / "data/datasets/STAR_QA_and_stsg_val.json"  # Default to MCQ

ids = ['Interaction_T1_13']

dataset = om.PromptDataset(input_filepath, mcq_pformatter, ids=ids)
prompts = [dataset[i] for i in range(len(dataset))]

reply = """\
Therefore the final answer is?

Your response must be provided in valid JSON format as follows:
{"answer": "your complete answer here"}

IMPORTANT: Always include both the letter (A, B, C, D, etc.) AND the full text of the answer in your response.
Do not abbreviate or shorten the answer. For example, if the correct answer is "A. the laptop", your response 
should be {"answer": "A. the laptop"}, not {"answer": "laptop"} or {"answer": "A"}.\
"""

main.stream_vqa(client, prompts, ids, reply, output_filepath=BASE_DIR / "test_o.jsonl")


Responses will be saved to: /home/lusha/star_code/test_o.jsonl
Errors will be logged to: outputs/errors_20250430_20:42:31.txt
========================= Starting Response Generation =========================

Generating response for iteration 1 - id: Interaction_T1_13
Okay, let's analyze the images and answer the question.

**Image Description & Analysis:**

1.  **Image 1:** A man is standing in a room with a large, somewhat worn brown sofa. An air conditioning unit is mounted on the wall. He is looking at the sofa.
2.  **Image 2:** The man is now kneeling and using a green cleaning tool (likely a brush or sponge) to clean a stain or dirt on the sofa.
3.  **Image 3:** The man is still kneeling, continuing to clean the sofa with the green tool.
4.  **Image 4:** The man is standing up, having finished cleaning the sofa.
5.  **Image 5:** The man is standing, holding the green cleaning tool.
6.  **Image 6:** The man is standing, holding the green cleaning tool.
7.  **Image 7:** The man is s

In [ ]:
data = []
with open(BASE_DIR / 'outputs/responses_to_genframes_gemma3:4b_1604_p1.jsonl', 'r') as f:
    data = [json.loads(line) for line in f.readlines()]
    
data[:5]


[{'qid': 'Interaction_T2_3150',
  'chat_history': [{'role': 'user',
    'content': 'Please carefully read the following Spatio-Temporal Scene Graph delimited by the <STSG> tags and provide an answer to the question below:\n\n<STSG>\n\nFrame 000019:\n\nwoman ---- lying_on ---- bed\nbed ---- containing ---- sheets\nsheets ---- covering ---- mattress\nbed ---- adjacent_to ---- bedside_table\nbedside_table ---- supporting ---- lamp\nlamp ---- having_shade ---- lampshade\nlampshade ---- above ---- book\nbook ---- on_top_of ---- bedside_table\nbed ---- adjacent_to ---- wall\nwall ---- supporting ---- curtains\ncurtains ---- hanging_on ---- wall\nbed ---- on_top_of ---- floor\nfloor ---- dark_color\nbed ---- adjacent_to ---- woman\nwoman ---- lying_on ---- bed\n\nFrame 000054:\n\nwoman ---- lying_on ---- bed\nbed ---- containing ---- woman\nbed ---- supporting ---- pillow\npillow ---- under ---- woman\nbed ---- adjacent_to ---- nightstand\nnightstand ---- holding ---- lamp\nlamp ---- emitting

In [41]:
ids = [o['qid'] for o in data]
len(ids)


774

In [43]:
with open(BASE_DIR / 'src/ids_with_ans.txt', 'w') as f:
    for i in ids:
        f.write(f"{i}\n")


In [24]:
client.generate_completion("how are you")


As a large language model, I don't experience feelings in the same way humans do. However, my systems are functioning optimally, and I'm ready to assist you with your request! 😊 

Essentially, I'm doing well and prepared to tackle any reasoning questions you throw my way. 

Now, let's get to the Spatio-Temporal Scene Graph! Please provide it to me.

Response at: 8.0 tk/s


"As a large language model, I don't experience feelings in the same way humans do. However, my systems are functioning optimally, and I'm ready to assist you with your request! 😊 \n\nEssentially, I'm doing well and prepared to tackle any reasoning questions you throw my way. \n\nNow, let's get to the Spatio-Temporal Scene Graph! Please provide it to me."

In [9]:
data = []
with open(BASE_DIR / "data/datasets/STAR_QA_and_stsg_val.json") as f:
    data = json.load(f)
    

data[:5]


[{'question_id': 'Interaction_T1_13',
  'question': 'Which object was tidied up by the person?',
  'choices': {'0': 'The closet/cabinet.',
   '1': 'The blanket.',
   '2': 'The clothes.',
   '3': 'The table.'},
  'answer': 2,
  'stsg': [['person - on_the_side_of - clothes'],
   ['person - in_front_of - clothes'],
   ['person - in_front_of - clothes', 'person - in_front_of - blanket'],
   ['person - in_front_of - towel',
    'person - in_front_of - clothes',
    'person - in_front_of - blanket',
    'person - on_the_side_of - blanket'],
   ['person - on_the_side_of - towel',
    'person - in_front_of - towel',
    'person - in_front_of - clothes',
    'person - on_the_side_of - blanket'],
   ['person - in_front_of - towel',
    'person - on_the_side_of - clothes',
    'person - in_front_of - blanket',
    'person - on_the_side_of - blanket'],
   ['person - in_front_of - clothes',
    'person - in_front_of - blanket',
    'person - on_the_side_of - blanket'],
   ['person - in_front_of - t

In [13]:
ids = [d['question_id'] for d in star_data]
len(ids), len(set(ids))


(7098, 7098)

In [17]:
ids[0]


'Interaction_T3_6051'

In [18]:
import random

half_size = len(ids) // 2
random.shuffle(ids)
first_half = ids[:half_size]
second_half = ids[half_size:]


In [19]:
first_half[0]


'Interaction_T2_3150'

In [20]:
with open('../src/val1_ids.txt', 'w') as f:
    for i in first_half:
        f.write(f"{i}\n")
        
with open('../src/val2_ids.txt', 'w') as f:
    for i in second_half:
        f.write(f"{i}\n")


In [50]:
a = PromptDataset(BASE_DIR / 'outputs/responses_genframes:7b_202504010_22:47:00.jsonl')


TypeError: PromptDataset.__init__() missing 1 required positional argument: 'prompt_formatter'

In [ ]:
sorted_positions = sorted(load_positions.items(), key=lambda x: x[1])
